In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install segmentation-models-pytorch
!pip install wandb
!pip install --upgrade wandb
!pip install torchmetrics
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import cv2
from torch.utils.data import Dataset
import torch
from PIL import Image
import matplotlib.pyplot as plt
from albumentations.pytorch import ToTensorV2
import albumentations as A
from tqdm import tqdm
import segmentation_models_pytorch as smp
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CyclicLR
import torch.nn.functional as F
from torchmetrics import JaccardIndex 
import wandb
from kaggle_secrets import UserSecretsClient

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 58.1 MB/s eta 0:00:00:00:01
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=821d01968e0c34dbb0d5058174ed080a8fbecfc269f706ee53dd98737960aa04
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=d80d7b5d0e586b06ad2a92136324bcc6bfbb32b139601bd6cc960e6ec3945c81
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-p

In [2]:
data_dir = ['../input/lyft-udacity-challenge/data'+i+'/data'+i for i in ['A']]
test_data_dir = ["/kaggle/input/lyft-udacity-challenge/dataC/dataC"]

!jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000.0
# wandb.init(project="SceneSegmentation")
# user_secrets = UserSecretsClient()
# wandb.login(key="21b310a8fc1549a4192f649372483fe3bafe0c7a", relogin=True)
# # my_secret = user_secrets.get_secret("21b310a8fc1549a4192f649372483fe3bafe0c7a") 


[I 13:28:50.551 NotebookApp] [nb_conda_kernels] enabled, 1 kernels found

  _   _          _      _
 | | | |_ __  __| |__ _| |_ ___
 | |_| | '_ \/ _` / _` |  _/ -_)
  \___/| .__/\__,_\__,_|\__\___|
       |_|
                       
Read the migration plan to Notebook 7 to learn about the new features and the actions to take if you are using extensions.

https://jupyter-notebook.readthedocs.io/en/latest/migrate_to_notebook7.html

Please note that updating to Notebook 7 might break some of your extensions.

[I 13:28:50.877 NotebookApp] Registered dataproc_jupyter_plugin server extension
jupyter_http_over_ws extension initialized. Listening on /http_over_websocket
[I 13:28:51.151 NotebookApp] Skipped non-installed server(s): bash-language-server, dockerfile-language-server-nodejs, javascript-typescript-langserver, jedi-language-server, julia-language-server, pyright, python-language-server, r-languageserver, sql-language-server, texlab, typescript-language-server, unified-language-server

In [32]:
# import torch, gc
# # gc.collect()
# torch.cuda.empty_cache()

In [3]:
data_dir

['../input/lyft-udacity-challenge/dataA/dataA']

In [4]:
class SegmentDataset(Dataset):
    def __init__(self,img_dir,transform = None):
        self.transforms = transform
        image_paths = [i+'/CameraRGB' for i in img_dir]
        seg_paths = [i+'/CameraSeg' for i in img_dir]
        self.images = []
        self.masks = []
        for i in image_paths:
            imgs = os.listdir(i)
            self.images.extend([i+'/'+img for img in imgs])
        for i in seg_paths:
            masks = os.listdir(i)
            self.masks.extend([i+'/'+mask for mask in masks])
    def __len__(self):
        return len(self.images)
    def __getitem__(self,index):
        img = cv2.imread(self.images[index])
        mask = cv2.imread(self.masks[index])
        aug = self.transforms(image=img,mask=mask)
        img = aug['image']
        mask = aug['mask']
        mask = torch.max(mask,dim=2)[0]
        return img,mask
        
        

In [5]:
# img = np.array(Image.open('/kaggle/input/lyft-udacity-challenge/dataA/dataA/CameraRGB/02_00_000.png'))

In [6]:
# import cv2
# img_arr = cv2.imread('/kaggle/input/lyft-udacity-challenge/dataA/dataA/CameraSeg/02_00_000.png')
# print(img_arr.shape)

In [7]:
# img_arr.shape

In [8]:
# t1 = A.Compose([
#     A.Resize(512,512),
#     A.augmentations.transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
#     ToTensorV2()
# ])


In [9]:
im = t1(image = img_arr)

NameError: name 't1' is not defined

In [10]:
pip install --upgrade wandb


Note: you may need to restart the kernel to use updated packages.


In [11]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
iou_method = JaccardIndex(task='multiclass',num_classes=23).to(DEVICE)
loss_fn = nn.CrossEntropyLoss()


In [12]:
class Segment:
    
#     DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
#     iou = JaccardIndex(task='multiclass',num_classes=23).to(DEVICE)
#     loss_fn = nn.CrossEntropyLoss()

    def __init__(self,image_dir,batch_size,transform):
        self.image_dir = image_dir
        self.batch_size = batch_size
        self.transform = transform
        self.train_size = 0.8
        self.num_epochs = 600
        self.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
        self.iou = JaccardIndex(task='multiclass',num_classes=23).to(self.DEVICE)
        self.lr = 0.1
        self.loss_fn = nn.CrossEntropyLoss()
#         self.optimizer = Adam(model.parameters(), lr=self.lr)
        self.scaler = torch.cuda.amp.GradScaler()
        self.student_loss_function = nn.KLDivLoss(reduction="mean")
#         wandb.login(key=my_secret)

    def split_data(self):
        data = SegmentDataset(self.image_dir,self.transform)
        train_size = int(self.train_size * data.__len__())
        test_size = data.__len__() - train_size
        shuffle,pin_memory = True,True
        train_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, test_size])
        train_batch = torch.utils.data.DataLoader(train_dataset, batch_size=self.batch_size, shuffle=shuffle, pin_memory=pin_memory)
        val_batch = torch.utils.data.DataLoader(test_dataset, batch_size=self.batch_size, shuffle=shuffle, pin_memory=pin_memory)
        test_dataset = SegmentDataset(test_data_dir,transform = self.transform)
        test_data_batch  = torch.utils.data.DataLoader(test_dataset, batch_size=self.batch_size, shuffle=shuffle, pin_memory=pin_memory)
        return train_batch,val_batch,test_data_batch
    
    def iou_cal(self,predictions, targets):
        iou = self.iou(predictions, targets)
        return iou
    
    @staticmethod
    def evaluate(model, dataloader):
        model.eval()
        losses = 0
        iou = 0
        print("Per Epoch Validation Phase ...")
    #     qbar = tqdm(range(len(val_dataloader)))
        for data, labels in dataloader:
            data = data.to(DEVICE)
            labels = labels.to(DEVICE)
            labels = labels.type(torch.long)

            pred = model(data)
            iou += iou_method(pred, labels)

            loss = loss_fn(pred, labels)
            losses += loss.item()
    #         qbar.update()

        return losses / len(dataloader),iou/len(dataloader)
    
    
    
    
    def train(self, student_model, teacher_model):
        wandb.finish()
        wandb.init(project="SceneSegmentation15")
        user_secrets = UserSecretsClient()
        wandb.login(key="21b310a8fc1549a4192f649372483fe3bafe0c7a", relogin=True)

        history = []
        train_batch, val_batch, test_data_batch = sel21b310a8fc1549a4192f649372483fe3bafe0c7af.split_data()

        # Initialize optimizers
        teacher_optimiser = Adam(teacher_model.parameters(), lr=self.lr)
        student_optimiser = Adam(student_model.parameters(), lr=self.lr)

        # Initialize schedulers
        teacher_scheduler = CyclicLR(teacher_optimiser, base_lr=0.001, max_lr=0.1, step_size_up=len(train_batch) // 2, mode='triangular', cycle_momentum=False)
        student_scheduler = CyclicLR(student_optimiser, base_lr=0.001, max_lr=0.1, step_size_up=len(train_batch) // 2, mode='triangular', cycle_momentum=False)

        # Convert models to float16
        student_model = student_model.to(self.DEVICE)
        teacher_model = teacher_model.to(self.DEVICE)

        for epoch in range(self.num_epochs):
            print(f"--------------EPOCH--------------- {epoch}")
            loop = tqdm(enumerate(train_batch), total=len(train_batch))
            train_iou = 0

            student_model.train()
            teacher_model.train()

            for batch_idx, (data, targets) in enumerate(train_batch):
                data = data.to(self.DEVICE)
                targets = targets.to(self.DEVICE)
                targets = targets.type(torch.long)

#                 with autocast():
                # Forward pass
                teacher_predictions = teacher_model(data)
                teacher_loss = self.loss_fn(teacher_predictions, targets)

                student_predictions = student_model(data)
                student_ce_loss = self.loss_fn(student_predictions, targets)

                student_kd_loss = self.student_loss_function(F.log_softmax(student_predictions, dim=1), F.softmax(teacher_predictions.detach(), dim=1))
                student_loss = student_kd_loss + student_ce_loss

                # Compute IOU
                train_iou = self.iou_cal(student_predictions, targets)

                # Zero gradients
                student_optimiser.zero_grad()
                teacher_optimiser.zero_grad()

                # Backward pass
                self.scaler.scale(student_loss).backward()
                self.scaler.step(student_optimiser)
                self.scaler.update()

                self.scaler.scale(teacher_loss).backward()
                self.scaler.step(teacher_optimiser)
                self.scaler.update()

                # Step schedulers
                teacher_scheduler.step()
                student_scheduler.step()
                early_stopping = EarlyStopping(patience=8, verbose=True)
                early_stopping(student_loss.item(), student_model)
                if early_stopping.early_stop:
                    print("Early stopping")
                    break

#                 print(f"Epoch: {epoch}, Teacher Train loss: {teacher_loss.item():.3f}, Student Train loss: {student_loss.item():.3f}, Train IOU: {train_iou}")

                history.append({
                    "Epoch": epoch, 
                    "teacher_train_loss": teacher_loss.item(),
                    "student_train_loss": student_loss.item(),
                    "train_iou": train_iou
                })

            # Evaluate training and validation losses
            train_loss, train_iou = self.evaluate(student_model, train_batch)
            val_loss, val_iou = self.evaluate(student_model, val_batch)

            history.append({
                'train_iou': train_iou,
                'train_loss': train_loss,
                'val_iou': val_iou,
                'val_loss': val_loss
            })

            print(f"Train Loss: {train_loss:.3f}, Train IOU: {train_iou:.3f}, Val Loss: {val_loss:.3f}, Val IOU: {val_iou:.3f}")
            wandb.log({"Train IOU":train_iou,"Val IOU":val_iou,"Train Loss":train_loss,"Val Loss":val_loss})
            wandb.save('/kaggle/working/*')
    

In [13]:
# import os
# os.remove('/kaggle/working/mask3.png')

In [14]:
# teacher_model

teacher_model = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=23,                      # model output channels (number of classes in your dataset)
)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 233MB/s] 


In [15]:
# student_model
student_model = smp.Unet(
    encoder_name="resnet18",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=23,                      # model output channels (number of classes in your dataset)
)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 267MB/s]


In [17]:
batch_size=1
t1 = A.Compose([
    A.Resize(512,512),
    A.augmentations.transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])


In [18]:
import numpy as np
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint_LS.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
#         if self.verbose:
#             self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [19]:
segemtation = Segment(data_dir,batch_size,t1)

In [ ]:
segemtation.train(student_model,teacher_model)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


--------------EPOCH--------------- 0


...


  0%|          | 0/800 [05:53<?, ?it/s]


Per Epoch Validation Phase ...
Per Epoch Validation Phase ...
Train Loss: 0.131, Train IOU: 0.646, Val Loss: 0.181, Val IOU: 0.620
--------------EPOCH--------------- 120


  0%|          | 0/800 [05:56<?, ?it/s]


Per Epoch Validation Phase ...
Per Epoch Validation Phase ...
Train Loss: 0.157, Train IOU: 0.650, Val Loss: 0.210, Val IOU: 0.623
--------------EPOCH--------------- 121



  0%|          | 0/800 [06:00<?, ?it/s]


In [ ]:
# !jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000.0

In [9]:
# Number of parameters

total_params = sum(p.numel() for p in teacher_model.parameters())
print(f"Number of parameters in Teacher model: {total_params}")

total_params = sum(p.numel() for p in student_model.parameters())
print(f"Number of parameters in Student model: {total_params}")


Number of parameters in Teacher model: 24439559
Number of parameters in Student model: 14331399


In [ ]:
t1 =  = A.Compose([
    A.Resize(512,512),
    A.augmentations.transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])


In [ ]:
data = SegmentDataset(data_dir,transform = t1)

In [7]:
iou = JaccardIndex(task='multiclass',num_classes=23)

In [ ]:
iou()

In [ ]:
train_batch,val_batch,test_data_batch = get_images(data_dir,transform =t1,batch_size=8)

In [ ]:

t1 = A.Compose([
    A.Resize(512,512),
    A.augmentations.transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])


In [ ]:
import torch
model1 = teacher_model #.load_state_dict(torch.load('/kaggle/working/checkpoint.pt',map_location='cpu'))
model1.load_state_dict(torch.load('/kaggle/working/checkpoint.pt'))
model1.eval()
# print(model)

In [ ]:
for x,y in test_data_batch:
    
#     print(x)
    fig , ax =  plt.subplots(3, 3, figsize=(18, 18))
    softmax = nn.Softmax(dim=1)
    preds = torch.argmax(softmax(model1(x)),axis=1).to('cpu')
    img1 = np.transpose(np.array(x[0,:,:,:].to('cpu')),(1,2,0))
    preds1 = np.array(preds[0,:,:])
    mask1 = np.array(y[0,:,:])
    img2 = np.transpose(np.array(x[1,:,:,:].to('cpu')),(1,2,0))
    preds2 = np.array(preds[1,:,:])
    mask2 = np.array(y[1,:,:])
    img3 = np.transpose(np.array(x[2,:,:,:].to('cpu')),(1,2,0))
    preds3 = np.array(preds[2,:,:])
    mask3 = np.array(y[2,:,:])
    ax[0,0].set_title('Image')
    ax[0,1].set_title('Prediction')
    ax[0,2].set_title('Mask')
    ax[1,0].set_title('Image')
    ax[1,1].set_title('Prediction')
    ax[1,2].set_title('Mask')
    ax[2,0].set_title('Image')
    ax[2,1].set_title('Prediction')
    ax[2,2].set_title('Mask')
    ax[0][0].axis("off")
    ax[1][0].axis("off")
    ax[2][0].axis("off")
    ax[0][1].axis("off")
    ax[1][1].axis("off")
    ax[2][1].axis("off")
    ax[0][2].axis("off")
    ax[1][2].axis("off")
    ax[2][2].axis("off")
    ax[0][0].imshow(img1)
    ax[0][1].imshow(preds1)
    ax[0][2].imshow(mask1)
    ax[1][0].imshow(img2)
    ax[1][1].imshow(preds2)
    ax[1][2].imshow(mask2)
    ax[2][0].imshow(img3)
    ax[2][1].imshow(preds3)
    ax[2][2].imshow(mask3)   
    break


In [ ]:
for img,mask in train_batch:
    img1 = np.transpose(img[0,:,:,:],(1,2,0))
    mask1 = np.array(mask[0,:,:])
    cv2.imwrite('mask1.png',mask1)
    img2 = np.transpose(img[1,:,:,:],(1,2,0))
    mask2 = np.array(mask[1,:,:])
    cv2.imwrite('mask2.png',mask2)
    img3 = np.transpose(img[2,:,:,:],(1,2,0))
    mask3 = np.array(mask[2,:,:])
    cv2.imwrite('mask3.png',mask3)
    fig , ax =  plt.subplots(3, 2, figsize=(18, 18))
    ax[0][0].imshow(img1)
    ax[0][1].imshow(mask1)
    ax[1][0].imshow(img2)
    ax[1][1].imshow(mask2)
    ax[2][0].imshow(img3)
    ax[2][1].imshow(mask3)
    break

In [ ]:
mas = cv2.imread('/kaggle/working/mask1.png')

In [ ]:
pip install torchsummary 

In [ ]:
# model1 = smp.Unet(
#     encoder_name="resnet18",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
#     in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#     classes=23,                      # model output channels (number of classes in your dataset)
# )

In [ ]:
total_params = sum(p.numel() for p in model1.parameters())
print(f"Number of parameters: {total_params}")total_params = sum(p.numel() for p in model1.parameters())
print(f"Number of parameters: {total_params}")

In [ ]:
teacher_model = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=23,                      # model output channels (number of classes in your dataset)
)

In [ ]:
student_model = smp.Unet(
    encoder_name="resnet18",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=23,                      # model output channels (number of classes in your dataset)
)

In [ ]:
# teacher_loss_function = nn.CrossEntropyLoss()
# student_loss_function = nn.KLDivLoss(reduction="batchmean")

In [ ]:
teacher_total_params = sum(p.numel() for p in teacher_model.parameters())
print(f"Number of Teacher parameters: {teacher_total_params}")
student_total_params = sum(p.numel() for p in student_model.parameters())
print(f"Number of Student parameters: {student_total_params}")

In [ ]:
len('24439559')

In [ ]:
LEARNING_RATE = 0.1
num_epochs = 10
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)
scaler = torch.cuda.amp.GradScaler()
teacher_loss_function = nn.CrossEntropyLoss()
student_loss_function = nn.KLDivLoss(reduction="batchmean")
iou = JaccardIndex(task='multiclass',num_classes=23)

In [ ]:
import numpy as np
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
model = model.to(DEVICE,dtype=torch.float32)

In [ ]:
# loss_fn = nn.CrossEntropyLoss()
# optimizer = Adam(model.parameters(), lr=LEARNING_RATE)
# scaler = torch.cuda.amp.GradScaler()
# teacher_loss_function = nn.CrossEntropyLoss()
# student_loss_function = nn.KLDivLoss(reduction="batchmean")

In [1]:
# IOU SCORE